# Single Trace Stage (STS)

In [ ]:
# Library & read Data
import _Library_HOSA
import time
import numpy as np
from _Library_HOSA import ClasseDataset
import pandas as pd
import gc
from obspy import UTCDateTime 

# paths of the dataset
hd = "/home/silvia/Desktop/Data/DETECT/Detect_all_data_extended.hdf5"
cs = "/home/silvia/Desktop/Data/DETECT/Detect_all_metadata_extended.csv"
# Dataset containing the traces (D.seismogram) and other metadata (D.metadata)
D = ClasseDataset()
D.leggi_custom_dataset(hd,cs)                                       
print("Dataset loaded")

In [ ]:


print("Start STS")

# uu used to generate the output in a CSV file, save the onset for each trace 
uu = pd.DataFrame.from_dict(D.metadata["trace_name"])              
uu["trace_P_arrival_sample"] = D.metadata["trace_P_arrival_sample"]

# p contains the various settings to be tested [statistic, filtertype, frequencies of filtering, window size of HOS, thersholds]
p = [[_Library_HOSA.S_4, "bandpass", [1,30], 300, [0.1,0.2,0.3,0.4]], 
     [_Library_HOSA.S_6, "bandpass", [1,30], 300, [0.1,0.2,0.3,0.4]]]
post_origin = 8

# used to generate a string that report the used statistic
names = ["S_4", "S_n"]                                                     
indi = 0
Tempo_Inizio = time.perf_counter()

# cycle on settings
for stat, filt, freq, wind, th in p:                                
    for ii in range(10):
         # free some memory, if needed
        gc.collect()  

    # key of the final dictionary
    string_key = f"stat: {str(names[indi])} type_filter: {filt} filter freq: {freq} window_width: {wind} tresh:"    
    print("I am working on impostation: ",hd,"\n", string_key)
    # I use different threshold for same setting
    ons_th = [[] for i in range(len(th)) ]                             
    ons_max = []
    
    for i in range(len(D.sismogramma)): 
        # register the sample corresponding to the event origin
        or_s =  int((UTCDateTime(D.metadata["source_origin_time"][i])- UTCDateTime(D.metadata["trace_start_time"][i]))*D.metadata["sampling_rate"][i])
        try:
            inizio = or_s-wind if or_s-wind > 0  else 0
            fine = inizio + post_origin*int(D.metadata["sampling_rate"][i])
            # I extract the portion of the waveform from the arrival until post_origin seconds after
            sig = _Library_HOSA.freq_filter(D.sismogramma[i][inizio:fine], D.metadata["sampling_rate"][i], freq, type_filter= filt)
            # returns onsets for various th_s, CF (HOS derivative), onset corresponding to th=1, lower_bound, HOS
            onset_th, diff, onset_max,lower_bound,hoss  = _Library_HOSA.get_onset_4(sig, wind, threshold=th, statistics= stat) 
            for j in range(len(th)):
                ons_th[j].append(onset_th[j] + or_s-wind)
            ons_max.append(onset_max + or_s-wind)
        except Exception as e: 
            print(e,"\nException occurred to trace", i)
            for j in range(len(th)):
                ons_th[j].append(9*10**10)
            ons_max.append(9*10**10)


    for j in range(len(th)):
        ons_th_tmp = np.array(ons_th[j])
        uu = pd.concat([uu,pd.DataFrame.from_dict({f"{string_key}_ons_th={th[j]}":ons_th_tmp})],axis=1)

    ons_max = np.array(ons_max)
    uu = pd.concat([uu,pd.DataFrame.from_dict({f"{string_key}_ons_max":ons_max})],axis=1)
    uu.to_csv("/home/silvia/Desktop/ONSET_HOS/ONSET_DETECT_whole.csv",index=False)
    indi +=1
print("\nAlgoritmh ran for ", time.perf_counter()-Tempo_Inizio, " to perform ", len(p), "tests on ", len(D.sismogramma)," traces each test")


# Multi Channel Stage

In [4]:
# Maximum allowed distance for clustering for each array (dmaxs = )
Dtmax = {'01': 100.27461126600004,
 '02': 111.27852548656281,
 '03': 188.8111269184575,
 '04': 218.25948507204254,
 '05': 140.40299960421459,
 '06': 114.23061990103015,
 '07': 145.60860570823075,
 '08': 122.52270975189279,
 '09': 137.45071610675376,
 '10': 114.12779094166119,
 '11': 139.3207171169968,
 '12': 211.48913707991554,
 '13': 198.34712908610084,
 '14': 134.23706918622923,
 '15': 88.35067937679354,
 '16': 148.98351384624613,
 '17': 145.07185770686453,
 '18': 150.0267067336913,
 '19': 252.46817648022636,
 '20': 121.88603567643541}

In [ ]:
import pandas as pd
import numpy as np
from _Library_HOSA import cluster_agg_max_distance, accept_cluster
times_hos = pd.read_csv("/home/silvia/Desktop/ONSET_HOS/ONSET_DETECT_whole.csv")     # onset deriving from STS
times_hos["Accept"] = "FALSE"

BEST_key = 'stat: S_4 type_filter: bandpass filter freq: [1, 30] window_width: 300 tresh:_ons_th=0.1' # Best setting in our studycase
all_best_keys = ['stat: S_4 type_filter: bandpass filter freq: [1, 30] window_width: 300 tresh:_ons_th=0.1',
                 'stat: S_4 type_filter: bandpass filter freq: [1, 30] window_width: 300 tresh:_ons_th=0.2',
                 'stat: S_4 type_filter: bandpass filter freq: [1, 30] window_width: 300 tresh:_ons_th=0.3',
                 'stat: S_4 type_filter: bandpass filter freq: [1, 30] window_width: 300 tresh:_ons_th=0.4',
                 'stat: S_4 type_filter: bandpass filter freq: [1, 30] window_width: 300 tresh:_ons_max']
std_eq = 15 

uu = []

indi_concordi = list(times_hos[all_best_keys][(times_hos[all_best_keys].std(axis=1)<=std_eq) & (times_hos[BEST_key]>0)].index) # check different thresholds are in accordance
times_concordi = times_hos[["trace_name"]+all_best_keys].iloc[indi_concordi,:]

ev_list = np.array([s[:12] for s in times_concordi["trace_name"]])
ev_uniq = list(set(ev_list))
ev_uniq.sort()

for ev in ev_uniq:
    tmp = times_concordi[(ev_list==ev)]                         # select a single event
    arr_list = np.array([s[16:18] for s in tmp["trace_name"]])  # select a single array for each event (tipical arr_list=["01", "01", "10"..] )
    arr_uniq = list(set(arr_list))
    arr_uniq.sort()
    for arr in arr_uniq:
        tmp_2 = tmp[(arr_list==arr)]                            # select a single array for each event          
        picks = tmp_2[BEST_key]

        if len(picks) <= 2:
            # Reject
            times_hos["Accept"][picks.index] = "FALSE"


        if len(picks) == 3:
            # Check max - min
            if (picks.max()-picks.min()) < Dtmax[arr]:
                times_hos["Accept"][picks.index] = "TRUE"

        if len(picks) > 3:
            picks = picks.sort_values()
            picks_l = list(picks.values)
            if len(picks) == 7:
                uu.append(picks_l+[arr])
            s,e = cluster_agg_max_distance(picks_l,dmax=Dtmax[arr])
            # for i in range(len(s)):
            #     print(picks_l[s[i]:e[i]+1])
            indi = accept_cluster(s,e)

            if indi >=0:
                times_hos["Accept"][picks[s[indi]:e[indi]+1].index] = "TRUE"

                
times_hos.to_csv(f"/home/silvia/Desktop/ONSET_HOS/ONSET_DETECT_whole_checked(std{std_eq}_tolerance40)_New.csv", index=False)